In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [17]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
#     aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
#                                                                       labels=actions_labels))
    ################################################ s', r = env(s, a)
    next_states_logits, Qs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits,
                                                                   labels=next_states_labels))
    #################################################### s'', Q' = env(s', a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_next_states_logits, nextQs_logits = Env(actions=next_actions_logits, states=next_states, # given
                                                 hidden_size=hidden_size, action_size=action_size, 
                                                 state_size=state_size, reuse=True)
    #################################################### s'', Q' = env(~s', ~a')
    next_actions_logits_ = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                               reuse=True)
    next_next_states_logits_, nextQs_logits_ = Env(actions=next_actions_logits_, states=next_states_logits, # pred
                                                   hidden_size=hidden_size, action_size=action_size, 
                                                   state_size=state_size, reuse=True)
    ##################################################### GAN
    Qs = tf.reshape(Qs_logits, shape=[-1])
    nextQs = tf.reshape(nextQs_logits, shape=[-1]) * (1-dones)
    nextQs_ = tf.reshape(nextQs_logits_, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(nextQs+nextQs_)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(nextQs+nextQs_)/2, # GAN
#                                                                     labels=tf.ones_like(rates))) # max
    ########################################################## Qlearning
    targetQs = rewards + (nextQs+nextQs_)/2
    eloss += tf.reduce_mean(tf.square(Qs - targetQs))
    ######################################################### Curosity=intrinsic reward
    rloss = tf.square(nextQs-nextQs_)
    sloss = tf.reduce_sum(tf.square(next_next_states_logits-next_next_states_logits_), axis=1)
    #### Q/r loss
    mean, var = tf.nn.moments(x=rloss,axes=[0])
    std = tf.sqrt(var)
    rloss_norm = (rloss - mean)/ std 
    #### State loss
    mean, var = tf.nn.moments(x=sloss,axes=[0])
    std = tf.sqrt(var)
    sloss_norm = (sloss - mean)/ std
    #### Intrinsic_rewards
    intrinsic_rewards = sloss_norm + rloss_norm
    neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                              labels=actions_labels)
    aloss = tf.reduce_mean(neg_log_prob*intrinsic_rewards)
    return actions_logits, aloss, eloss

In [18]:
def model_opt(a_loss, e_loss, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
    return a_opt, e_opt

In [24]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt = model_opt(a_loss=self.a_loss, e_loss=self.e_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [25]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [26]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [27]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [28]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [29]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch= [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:19.0000 R:19.0000 rate:0.0380 aloss:0.0475 eloss:3.5966 exploreP:0.9981
Episode:1 meanR:19.0000 R:19.0000 rate:0.0380 aloss:0.0406 eloss:3.6127 exploreP:0.9962
Episode:2 meanR:30.0000 R:52.0000 rate:0.1040 aloss:0.0166 eloss:3.4886 exploreP:0.9911
Episode:3 meanR:30.7500 R:33.0000 rate:0.0660 aloss:0.0034 eloss:3.4279 exploreP:0.9879
Episode:4 meanR:28.8000 R:21.0000 rate:0.0420 aloss:0.0182 eloss:3.4182 exploreP:0.9858
Episode:5 meanR:26.3333 R:14.0000 rate:0.0280 aloss:0.0397 eloss:3.4279 exploreP:0.9845
Episode:6 meanR:26.1429 R:25.0000 rate:0.0500 aloss:0.0093 eloss:3.4136 exploreP:0.9820
Episode:7 meanR:26.1250 R:26.0000 rate:0.0520 aloss:0.0082 eloss:3.4011 exploreP:0.9795
Episode:8 meanR:25.1111 R:17.0000 rate:0.0340 aloss:-0.0077 eloss:3.2314 exploreP:0.9779
Episode:9 meanR:25.0000 R:24.0000 rate:0.0480 aloss:0.0026 eloss:3.2453 exploreP:0.9756
Episode:10 meanR:24.1818 R:16.0000 rate:0.0320 aloss:-0.0039 eloss:3.2867 exploreP:0.9740
Episode:11 meanR:24.1667 R:24

Episode:92 meanR:20.6667 R:33.0000 rate:0.0660 aloss:-2.8872 eloss:1.5379 exploreP:0.8269
Episode:93 meanR:20.6277 R:17.0000 rate:0.0340 aloss:-2.1931 eloss:1.4708 exploreP:0.8255
Episode:94 meanR:20.8105 R:38.0000 rate:0.0760 aloss:-2.5496 eloss:1.4679 exploreP:0.8224
Episode:95 meanR:20.8854 R:28.0000 rate:0.0560 aloss:-2.9080 eloss:1.4697 exploreP:0.8201
Episode:96 meanR:20.9175 R:24.0000 rate:0.0480 aloss:-3.4937 eloss:1.4617 exploreP:0.8182
Episode:97 meanR:20.9592 R:25.0000 rate:0.0500 aloss:-3.5820 eloss:1.4478 exploreP:0.8162
Episode:98 meanR:21.0202 R:27.0000 rate:0.0540 aloss:-3.5107 eloss:1.4443 exploreP:0.8140
Episode:99 meanR:20.9600 R:15.0000 rate:0.0300 aloss:-4.0596 eloss:1.4760 exploreP:0.8128
Episode:100 meanR:20.8800 R:11.0000 rate:0.0220 aloss:-3.2611 eloss:1.4348 exploreP:0.8119
Episode:101 meanR:20.8800 R:19.0000 rate:0.0380 aloss:-3.7741 eloss:1.4522 exploreP:0.8104
Episode:102 meanR:20.5700 R:21.0000 rate:0.0420 aloss:-4.3041 eloss:1.4359 exploreP:0.8087
Episode

Episode:182 meanR:18.2100 R:10.0000 rate:0.0200 aloss:-16.8822 eloss:1.3254 exploreP:0.7067
Episode:183 meanR:18.1500 R:11.0000 rate:0.0220 aloss:-21.6724 eloss:1.3398 exploreP:0.7059
Episode:184 meanR:18.0800 R:8.0000 rate:0.0160 aloss:-19.7466 eloss:1.3305 exploreP:0.7053
Episode:185 meanR:18.0600 R:14.0000 rate:0.0280 aloss:-18.7932 eloss:1.3416 exploreP:0.7044
Episode:186 meanR:17.8000 R:14.0000 rate:0.0280 aloss:-20.2055 eloss:1.3343 exploreP:0.7034
Episode:187 meanR:17.7700 R:10.0000 rate:0.0200 aloss:-21.5488 eloss:1.3275 exploreP:0.7027
Episode:188 meanR:18.1400 R:51.0000 rate:0.1020 aloss:-20.4090 eloss:1.3301 exploreP:0.6992
Episode:189 meanR:17.9200 R:12.0000 rate:0.0240 aloss:-16.1767 eloss:1.3614 exploreP:0.6984
Episode:190 meanR:17.7500 R:9.0000 rate:0.0180 aloss:-15.0523 eloss:1.3456 exploreP:0.6977
Episode:191 meanR:17.9600 R:42.0000 rate:0.0840 aloss:-19.3809 eloss:1.3541 exploreP:0.6949
Episode:192 meanR:17.7200 R:9.0000 rate:0.0180 aloss:-21.9395 eloss:1.3176 explore

Episode:272 meanR:16.9200 R:15.0000 rate:0.0300 aloss:-56.8195 eloss:1.2950 exploreP:0.6094
Episode:273 meanR:16.9300 R:18.0000 rate:0.0360 aloss:-52.8669 eloss:1.4037 exploreP:0.6084
Episode:274 meanR:16.7800 R:14.0000 rate:0.0280 aloss:-55.5060 eloss:1.2675 exploreP:0.6075
Episode:275 meanR:16.8000 R:13.0000 rate:0.0260 aloss:-62.5901 eloss:1.2823 exploreP:0.6068
Episode:276 meanR:16.5400 R:12.0000 rate:0.0240 aloss:-55.7309 eloss:1.2772 exploreP:0.6060
Episode:277 meanR:16.5300 R:14.0000 rate:0.0280 aloss:-62.3954 eloss:1.2740 exploreP:0.6052
Episode:278 meanR:16.3800 R:10.0000 rate:0.0200 aloss:-65.5616 eloss:1.2900 exploreP:0.6046
Episode:279 meanR:16.3800 R:12.0000 rate:0.0240 aloss:-63.5018 eloss:1.2803 exploreP:0.6039
Episode:280 meanR:16.4000 R:20.0000 rate:0.0400 aloss:-59.5850 eloss:1.2765 exploreP:0.6027
Episode:281 meanR:16.4100 R:15.0000 rate:0.0300 aloss:-67.5752 eloss:1.2930 exploreP:0.6018
Episode:282 meanR:16.4700 R:16.0000 rate:0.0320 aloss:-68.9410 eloss:1.2757 expl

Episode:361 meanR:14.7400 R:10.0000 rate:0.0200 aloss:-122.2255 eloss:1.2616 exploreP:0.5359
Episode:362 meanR:14.6900 R:16.0000 rate:0.0320 aloss:-126.6697 eloss:1.2588 exploreP:0.5351
Episode:363 meanR:14.7200 R:13.0000 rate:0.0260 aloss:-123.6846 eloss:1.2581 exploreP:0.5344
Episode:364 meanR:14.7700 R:14.0000 rate:0.0280 aloss:-123.5804 eloss:1.3192 exploreP:0.5336
Episode:365 meanR:14.6800 R:12.0000 rate:0.0240 aloss:-123.7500 eloss:1.2626 exploreP:0.5330
Episode:366 meanR:14.6300 R:15.0000 rate:0.0300 aloss:-121.1644 eloss:1.2772 exploreP:0.5322
Episode:367 meanR:14.5600 R:10.0000 rate:0.0200 aloss:-127.0644 eloss:1.2653 exploreP:0.5317
Episode:368 meanR:14.5700 R:13.0000 rate:0.0260 aloss:-125.3229 eloss:1.2559 exploreP:0.5310
Episode:369 meanR:14.4800 R:10.0000 rate:0.0200 aloss:-143.1531 eloss:1.2679 exploreP:0.5305
Episode:370 meanR:14.5200 R:15.0000 rate:0.0300 aloss:-116.2584 eloss:1.2526 exploreP:0.5297
Episode:371 meanR:14.6000 R:18.0000 rate:0.0360 aloss:-123.0343 eloss:

...

Episode:1072 meanR:10.7400 R:9.0000 rate:0.0180 aloss:-577.1251 eloss:1.2138 exploreP:0.2349
Episode:1073 meanR:10.7400 R:10.0000 rate:0.0200 aloss:-514.8857 eloss:1.2327 exploreP:0.2347
Episode:1074 meanR:10.7200 R:10.0000 rate:0.0200 aloss:-466.4576 eloss:1.2557 exploreP:0.2344
Episode:1075 meanR:10.7100 R:9.0000 rate:0.0180 aloss:-535.4973 eloss:1.2912 exploreP:0.2342
Episode:1076 meanR:10.7400 R:13.0000 rate:0.0260 aloss:-557.2174 eloss:1.2237 exploreP:0.2339
Episode:1077 meanR:10.7200 R:11.0000 rate:0.0220 aloss:-405.5973 eloss:1.2768 exploreP:0.2337
Episode:1078 meanR:10.7000 R:10.0000 rate:0.0200 aloss:-609.8009 eloss:1.2132 exploreP:0.2335
Episode:1079 meanR:10.7400 R:13.0000 rate:0.0260 aloss:-545.6641 eloss:1.2310 exploreP:0.2332
Episode:1080 meanR:10.7000 R:10.0000 rate:0.0200 aloss:-582.3854 eloss:1.2260 exploreP:0.2330
Episode:1081 meanR:10.6800 R:9.0000 rate:0.0180 aloss:-634.4885 eloss:1.2031 exploreP:0.2328
Episode:1082 meanR:10.6800 R:13.0000 rate:0.0260 aloss:-510.791

Episode:1160 meanR:10.5400 R:12.0000 rate:0.0240 aloss:-553.6688 eloss:1.2204 exploreP:0.2149
Episode:1161 meanR:10.5400 R:10.0000 rate:0.0200 aloss:-579.2558 eloss:1.2228 exploreP:0.2147
Episode:1162 meanR:10.5200 R:11.0000 rate:0.0220 aloss:-556.2090 eloss:1.2193 exploreP:0.2144
Episode:1163 meanR:10.5300 R:9.0000 rate:0.0180 aloss:-639.0641 eloss:1.2039 exploreP:0.2142
Episode:1164 meanR:10.5200 R:8.0000 rate:0.0160 aloss:-582.1917 eloss:1.2178 exploreP:0.2141
Episode:1165 meanR:10.5800 R:15.0000 rate:0.0300 aloss:-565.6942 eloss:1.3067 exploreP:0.2138
Episode:1166 meanR:10.5800 R:10.0000 rate:0.0200 aloss:-619.2054 eloss:1.2107 exploreP:0.2136
Episode:1167 meanR:10.5700 R:8.0000 rate:0.0160 aloss:-552.4023 eloss:1.2187 exploreP:0.2134
Episode:1168 meanR:10.5300 R:9.0000 rate:0.0180 aloss:-447.8632 eloss:1.3037 exploreP:0.2132
Episode:1169 meanR:10.5200 R:9.0000 rate:0.0180 aloss:-584.0001 eloss:1.2191 exploreP:0.2130
Episode:1170 meanR:10.5300 R:11.0000 rate:0.0220 aloss:-595.0805 

Episode:1248 meanR:10.7000 R:9.0000 rate:0.0180 aloss:-535.7791 eloss:1.2214 exploreP:0.1964
Episode:1249 meanR:10.6800 R:9.0000 rate:0.0180 aloss:-667.1105 eloss:1.2112 exploreP:0.1963
Episode:1250 meanR:10.7100 R:13.0000 rate:0.0260 aloss:-543.1015 eloss:1.2975 exploreP:0.1960
Episode:1251 meanR:10.7300 R:11.0000 rate:0.0220 aloss:-546.2986 eloss:1.2152 exploreP:0.1958
Episode:1252 meanR:10.7200 R:8.0000 rate:0.0160 aloss:-344.9019 eloss:1.3843 exploreP:0.1957
Episode:1253 meanR:10.7200 R:11.0000 rate:0.0220 aloss:-575.6982 eloss:1.2275 exploreP:0.1955
Episode:1254 meanR:10.7500 R:12.0000 rate:0.0240 aloss:-600.7852 eloss:1.2340 exploreP:0.1952
Episode:1255 meanR:10.7600 R:11.0000 rate:0.0220 aloss:-684.1279 eloss:1.2168 exploreP:0.1950
Episode:1256 meanR:10.7800 R:11.0000 rate:0.0220 aloss:-585.5193 eloss:1.2659 exploreP:0.1948
Episode:1257 meanR:10.8500 R:16.0000 rate:0.0320 aloss:-637.0479 eloss:1.2214 exploreP:0.1945
Episode:1258 meanR:10.8900 R:14.0000 rate:0.0280 aloss:-637.460

Episode:1336 meanR:10.6400 R:10.0000 rate:0.0200 aloss:-570.0057 eloss:1.2539 exploreP:0.1797
Episode:1337 meanR:10.6500 R:12.0000 rate:0.0240 aloss:-715.4634 eloss:1.2174 exploreP:0.1795
Episode:1338 meanR:10.6000 R:9.0000 rate:0.0180 aloss:-595.6305 eloss:1.2988 exploreP:0.1793
Episode:1339 meanR:10.5900 R:10.0000 rate:0.0200 aloss:-735.1143 eloss:1.2133 exploreP:0.1792
Episode:1340 meanR:10.6000 R:10.0000 rate:0.0200 aloss:-605.8715 eloss:1.2576 exploreP:0.1790
Episode:1341 meanR:10.6100 R:14.0000 rate:0.0280 aloss:-664.9400 eloss:1.2482 exploreP:0.1788
Episode:1342 meanR:10.6000 R:11.0000 rate:0.0220 aloss:-556.2632 eloss:1.2668 exploreP:0.1786
Episode:1343 meanR:10.6100 R:10.0000 rate:0.0200 aloss:-700.2875 eloss:1.2200 exploreP:0.1784
Episode:1344 meanR:10.6000 R:9.0000 rate:0.0180 aloss:-676.7072 eloss:1.2350 exploreP:0.1783
Episode:1345 meanR:10.6000 R:9.0000 rate:0.0180 aloss:-658.9147 eloss:1.2059 exploreP:0.1781
Episode:1346 meanR:10.6100 R:9.0000 rate:0.0180 aloss:-647.9789

Episode:1424 meanR:10.1800 R:18.0000 rate:0.0360 aloss:-704.5725 eloss:1.2266 exploreP:0.1652
Episode:1425 meanR:10.1900 R:11.0000 rate:0.0220 aloss:-737.4287 eloss:1.2520 exploreP:0.1650
Episode:1426 meanR:10.2100 R:11.0000 rate:0.0220 aloss:-738.4611 eloss:1.2173 exploreP:0.1649
Episode:1427 meanR:10.2200 R:9.0000 rate:0.0180 aloss:-804.9561 eloss:1.2277 exploreP:0.1647
Episode:1428 meanR:10.2000 R:9.0000 rate:0.0180 aloss:-648.2454 eloss:1.4500 exploreP:0.1646
Episode:1429 meanR:10.1300 R:9.0000 rate:0.0180 aloss:-644.6503 eloss:1.4399 exploreP:0.1645
Episode:1430 meanR:10.1500 R:11.0000 rate:0.0220 aloss:-777.9426 eloss:1.2225 exploreP:0.1643
Episode:1431 meanR:10.1700 R:10.0000 rate:0.0200 aloss:-718.1575 eloss:1.2266 exploreP:0.1641
Episode:1432 meanR:10.1600 R:9.0000 rate:0.0180 aloss:-748.4171 eloss:1.2142 exploreP:0.1640
Episode:1433 meanR:10.1400 R:12.0000 rate:0.0240 aloss:-800.2183 eloss:1.2229 exploreP:0.1638
Episode:1434 meanR:10.1500 R:13.0000 rate:0.0260 aloss:-736.8177

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
